In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/100-bird-species/'):
#     for filename in filenames:
#         print(os.path.join(filename))
for dirname in os.listdir('/kaggle/input/100-bird-species'):
    print(dirname)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

EfficientNetB0-525-(224 X 224)- 98.97.h5
valid
test
birds.csv
train


# Examins the data

In [6]:
# Import libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch dataset
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# PyTorch model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [7]:
# Data transform to convert data to a tensor and apply normalization

# augment train and validation dataset with RandomHorizontalFlip and RandomRotation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4234, 0.4089, 0.3468], std=[0.2358, 0.2295, 0.2207])
    ])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4234, 0.4089, 0.3468], std=[0.2358, 0.2295, 0.2207])
    ])

In [8]:
data_dir = '/kaggle/input/100-bird-species/'

In [9]:
# choose the training and test datasets
train_data = datasets.ImageFolder(f'{data_dir}train', transform=train_transform)
validation_data = datasets.ImageFolder(f'{data_dir}valid', transform=test_transform)
test_data = datasets.ImageFolder(f'{data_dir}test', transform=test_transform)

## Determine Image Mean

In [6]:
mean = 0.0
std = 0.0
total_samples = 0

for images, _ in trainloader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_samples += batch_samples

mean /= total_samples
std /= total_samples

print("Mean:", mean)
print("Standard Deviation:", std)

Mean: tensor([0.4234, 0.4089, 0.3468])
Standard Deviation: tensor([0.2358, 0.2295, 0.2207])


In [10]:
birds=list(train_data.class_to_idx.keys())

In [11]:
output_len= len(birds)

In [12]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(validation_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
images, labels = next(iter(trainloader))

fig, axes=plt.subplots(figsize=(50, 50), ncols=2)
for idx in np.arange(2):
    ax=axes[idx]
    image = images[idx].numpy().transpose((1,2,0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     image = std * image + mean
#     image = np.clip(image, 0, 1)
    ax.imshow(image)
    ax.set_title(birds[labels[idx].item()])

In [ ]:
image.shape

## Design Model

In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Conv2d(3, 64, 3, 1,1 ) # OP 224
        self.conv2=nn.Conv2d(64, 128, 3, 1,1 ) # OP 224
        self.conv3=nn.Conv2d(128, 256, 3, 1,1 ) # OP 224
        self.conv4=nn.Conv2d(256, 512, 3, 1,1 ) # OP 224
        
        self.pool = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(512,256)
        self.fc2 = nn.Linear(256, output_len)
        
        self.bc1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.bc2 = nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.bc3 = nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.bc4 = nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.avg = nn.AdaptiveAvgPool2d(output_size=(1,1))
        
        
        self.dropout=nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.bc1(self.conv1(x))))
        x = self.pool(F.relu(self.bc2(self.conv2(x))))
        x = self.pool(F.relu(self.bc3(self.conv3(x))))
        x = self.pool(F.relu(self.bc4(self.conv4(x))))
        
        # flatten the tensor
        x=self.avg(x)
        x = x.view(x.size(0), -1)
        
        x=self.dropout(F.relu(self.fc1(x)))
        x=self.fc2(x)
        x=F.log_softmax(x, dim=1)
        
        return x
model = Net()
model

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=525, bias=True)
  (bc1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bc2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bc3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bc4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (avg): AdaptiveAvgPool2d(output_size=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
)

In [31]:
# train_on_gpu = torch.cuda.is_available()
# if not train_on_gpu:
#     print('CUDA is not available.  Training on CPU ...')
# else:
#     model.cuda()
#     print('CUDA is available!  Training on GPU ...')
    
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

model.to(device)

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=525, bias=True)
  (bc1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bc2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bc3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bc4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (avg): AdaptiveAvgPool2d(output_size=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
)

In [15]:
device

device(type='cuda')

In [32]:
criterion = nn.CrossEntropyLoss()

optimizer=optim.Adam(model.parameters(), lr=0.003)

In [ ]:
epochs=100
valid_loss_min = np.Inf #used to track change

for epoch in range(1, epochs+1):
    
    ################
    ## train model
    ###############
    train_loss=0.0
    valid_loss=0.0
    accuracy=0.0
    model.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logps=model.forward(images)
        loss=criterion(logps, labels)
        
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
              
    # validation
    model.eval()
    with torch.no_grad():
        for images, labels in validloader:
            images, labels = images.to(device), labels.to(device)

            logps=model.forward(images)
            loss=criterion(logps, labels)
            ps=torch.exp(logps)
            top_k, top_class = ps.topk(1, dim=1)
            equality= top_class==labels.view(*top_class.shape)
            accuracy+=torch.mean(equality.type(torch.FloatTensor)).item()

            valid_loss+=loss.item()

    train_loss=train_loss/len(trainloader)
    valid_loss=valid_loss/len(validloader)
    accuracy=accuracy/len(validloader)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))

    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'bird_model_pers.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 5.989330 	Validation Loss: 5.687054 	 Accuracy: 0.011295
Validation loss decreased (inf --> 5.687054).  Saving model ...
Epoch: 2 	Training Loss: 5.796443 	Validation Loss: 5.484194 	 Accuracy: 0.019578
Validation loss decreased (5.687054 --> 5.484194).  Saving model ...
Epoch: 3 	Training Loss: 5.644553 	Validation Loss: 5.280692 	 Accuracy: 0.028614
Validation loss decreased (5.484194 --> 5.280692).  Saving model ...
Epoch: 4 	Training Loss: 5.388004 	Validation Loss: 4.862541 	 Accuracy: 0.076807
Validation loss decreased (5.280692 --> 4.862541).  Saving model ...
Epoch: 5 	Training Loss: 5.209293 	Validation Loss: 4.611605 	 Accuracy: 0.092997
Validation loss decreased (4.862541 --> 4.611605).  Saving model ...
Epoch: 6 	Training Loss: 5.064153 	Validation Loss: 4.494500 	 Accuracy: 0.094503
Validation loss decreased (4.611605 --> 4.494500).  Saving model ...
Epoch: 7 	Training Loss: 4.934724 	Validation Loss: 4.292379 	 Accuracy: 0.114458
Validation loss d

In [21]:
for epoch in range(1, epochs+1):
    
    ################
    ## train model
    ###############
    train_loss=0.0
    valid_loss=0.0
    accuracy=0.0
    model.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logps=model.forward(images)
        loss=criterion(logps, labels)
        
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
              
    # validation
    model.eval()
    with torch.no_grad():
        for images, labels in validloader:
            images, labels = images.to(device), labels.to(device)

            logps=model.forward(images)
            loss=criterion(logps, labels)
            ps=torch.exp(logps)
            top_k, top_class = ps.topk(1, dim=1)
            equality= top_class==labels.view(*top_class.shape)
            accuracy+=torch.mean(equality.type(torch.FloatTensor)).item()

            valid_loss+=loss.item()

    train_loss=train_loss/len(trainloader)
    valid_loss=valid_loss/len(validloader)
    accuracy=accuracy/len(validloader)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))

    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'bird_model_pers.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 5.088857 	Validation Loss: 4.525621 	 Accuracy: 0.103916
Validation loss decreased (4.660902 --> 4.525621).  Saving model ...
Epoch: 2 	Training Loss: 5.055562 	Validation Loss: 4.557027 	 Accuracy: 0.094503


KeyboardInterrupt: 

In [1]:
images.size(0)

NameError: name 'images' is not defined

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
images[33].shape

# Use Transfer Learning

In [25]:
from torchvision import datasets, transforms, models
pretrained = models.resnet50(pretrained=True)
for param in pretrained.parameters():
  param.requires_grad = False
pretrained

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [26]:
classifier = nn.Sequential(nn.Linear(2048,1024),
                           nn.Dropout(p=0.2),
                           nn.Linear(1024,output_len),
                           nn.LogSoftmax(dim=1))

In [31]:
pretrained.fc=classifier
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained.fc.parameters(), lr=0.03)

In [32]:
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

pretrained.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [33]:
epochs=5
valid_loss_min = np.Inf #used to track change
step=0
print_every=70


for epoch in range(1, epochs+1):
    
    ################
    ## train model
    ###############
    train_loss=0.0
    valid_loss=0.0
    accuracy=0.0
    pretrained.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logps=pretrained.forward(images)
        loss=criterion(logps, labels)
        
        loss.backward()
        optimizer.step()
        train_loss+=loss.item() # *images.size(0)
        
        # validation
#         if step%print_every==0:  
    pretrained.eval()
    with torch.no_grad():
        for images, labels in validloader:
            images, labels = images.to(device), labels.to(device)

            logps=pretrained.forward(images)
            loss=criterion(logps, labels)
            ps=torch.exp(logps)
            top_k, top_class = ps.topk(1, dim=1)
            equality= top_class==labels.view(*top_class.shape)
            accuracy+=torch.mean(equality.type(torch.FloatTensor)).item()

            valid_loss+=loss.item() # *images.size(0)

    train_loss=train_loss/len(trainloader)
    valid_loss=valid_loss/len(validloader)
    accuracy=accuracy/len(validloader)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))

    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(pretrained.state_dict(), 'bird_model.pt')
        valid_loss_min = valid_loss
    train_loss=0.0
    valid_loss=0.0
    accuracy=0.0

Epoch: 1 	Training Loss: 18001.101780 	Validation Loss: 7261.096303 	 Accuracy: 0.697289
Validation loss decreased (inf --> 7261.096303).  Saving model ...
Epoch: 2 	Training Loss: 14211.570658 	Validation Loss: 5679.569242 	 Accuracy: 0.725904
Validation loss decreased (7261.096303 --> 5679.569242).  Saving model ...
Epoch: 3 	Training Loss: 12308.089259 	Validation Loss: 5660.502397 	 Accuracy: 0.703690
Validation loss decreased (5679.569242 --> 5660.502397).  Saving model ...
Epoch: 4 	Training Loss: 11111.513369 	Validation Loss: 3939.102774 	 Accuracy: 0.737199
Validation loss decreased (5660.502397 --> 3939.102774).  Saving model ...
Epoch: 5 	Training Loss: 10117.043949 	Validation Loss: 4510.536478 	 Accuracy: 0.710467


In [14]:
for epoch in range(1, epochs+1):
    
    ################
    ## train model
    ###############
    train_loss=0.0
    valid_loss=0.0
    accuracy=0.0
    pretrained.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logps=pretrained.forward(images)
        loss=criterion(logps, labels)
        
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()*images.size(0)
        
        # validation
#         if step%print_every==0:  
    pretrained.eval()
    with torch.no_grad():
        for images, labels in validloader:
            images, labels = images.to(device), labels.to(device)

            logps=pretrained.forward(images)
            loss=criterion(logps, labels)
            ps=torch.exp(logps)
            top_k, top_class = ps.topk(1, dim=1)
            equality= top_class==labels.view(*top_class.shape)
            accuracy+=torch.mean(equality.type(torch.FloatTensor)).item()

            valid_loss+=loss.item()*images.size(0)

    train_loss=train_loss/len(trainloader)
    valid_loss=valid_loss/len(validloader)
    accuracy=accuracy/len(validloader)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))

    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(pretrained.state_dict(), 'bird_model.pt')
        valid_loss_min = valid_loss
    train_loss=0.0
    valid_loss=0.0
    accuracy=0.0

KeyboardInterrupt: 

In [24]:
epochs=30
for epoch in range(11, epochs+1):
    
    ################
    ## train model
    ###############
    train_loss=0.0
    valid_loss=0.0
    accuracy=0.0
    pretrained.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logps=pretrained.forward(images)
        loss=criterion(logps, labels)
        
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()*images.size(0)
        
    # validation
    pretrained.eval()
    with torch.no_grad():
        for images, labels in validloader:
            images, labels = images.to(device), labels.to(device)

            logps=pretrained.forward(images)
            loss=criterion(logps, labels)
            ps=torch.exp(logps)
            top_k, top_class = ps.topk(1, dim=1)
            equality= top_class==labels.view(*top_class.shape)
            accuracy+=torch.mean(equality.type(torch.FloatTensor)).item()

            valid_loss+=loss.item()*images.size(0)
        
    train_loss=train_loss/len(trainloader)
    valid_loss=valid_loss/len(validloader)
    accuracy=accuracy/len(validloader)
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))
    
    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(pretrained.state_dict(), 'bird_model.pt')
        valid_loss_min = valid_loss

NameError: name 'no_grad' is not defined